In [31]:
'''1. Remove foreign records to a 'foreign dataframe' and delete them from this df. - applicable for  address
2.Find whether all phone numbers are empty, if empty drop those rows (8708)
3.Column Phone matches the Phone number in preferred phone number
Transfer records with non-null values in Phone to a new dataframe with columns EID and phone
Find the count of records (124000)
4. Null values in Phone: (36000 records) Create a new df:Check the content in preferred phone 
and grasp the preferred phone and add it to df with columns eid and phone

Phone number formatting:

Format for 10 continuous numbers (USA), make numbers mentioned with USA international code +1 into 10 digits, ignore +1
Delete the records with more than 10 number, extension is mentioned
  -- If more than 10 numbers, if non usa number add it to a 'foreign dataframe' delete that number
  -- If extension is mentioned, add it to the 'extension dataframe'. Check whether the database has extension numbers mentioned.
If the number is less than 10 digits, compare the last 7 digits of the phone number from database

df_texPh  has original file
df_txPh has records after dropping records with no phone numbers
df_prfrPh has records with values in column 'Phone' 
df_Ph  has records with empty values in column 'Phone' 

There is a challenge. We cannot send more than 1000 EID in where clause of sql to access the records from database. 
So the idea of using sql embedded inside python is being dropped.
Instead we are going to do separate sql query joining every 1000 eid in 'or' clause.
The sql output from Database is going to be imported as a csv file
Solution:for i in range((len(eid) + n - 1) // n ):
            pid[i] = eid[i * n:(i + 1) * n]

Problem:  the EIDs from the csv file are not present in vi_vips_main table.
Is there a separate table for alumni? 
Solution: converting eid to all uppercase
'''
import pandas as pd
#import chardet

df_texPh = pd.read_csv('Texasxes.csv')
print("original count ",len(df_texPh))

df_texPh = df_texPh[['EID','Phone','Home Phone','Preferred Phone', 'Mobile','Work Phone','Other Phone']]
print("original count ",len(df_texPh))

df_txPh = df_texPh.dropna(how='all', subset=['Phone','Home Phone','Mobile','Work Phone','Other Phone'])
print("Count records after dropping records with no phone numbers : ",len(df_txPh))

df_prfrPh = df_txPh.dropna(subset=['Phone'])
print("Count records with values in column Phone : ",len(df_prfrPh))

#df = df_one[~df_one.index.isin(df_two.index)] Present in First DataFrame and Not in Second DataFrame
df_Ph = df_txPh[~df_txPh.index.isin(df_prfrPh.index)]
print("has records with empty values in column Phone : ",len(df_Ph))

del df_texPh, df_txPh, df_Ph

original count  2200
original count  2200
Count records after dropping records with no phone numbers :  2085
Count records with values in column Phone :  1765
has records with empty values in column Phone :  320


In [33]:
#df_prfrPh.to_csv(r'C:\Users\le5752\Documents\DB_Python\ph1.csv', index = False)
df_prfrPh1 = df_prfrPh[['EID','Phone']]
df_prfrPh1.to_csv(r'C:\Users\le5752\Documents\DB_Python\ph2.csv', index = False)
del df_prfrPh1
df_prfrPh1 = pd.read_csv('ph2.csv')
print(df_prfrPh1.head(11))

         EID           Phone
0     to3269  (281) 772-0155
1   0563numt      5124967336
2      ksn99  (361) 865-2686
3    jme3234      9152745699
4   0343rtia      4258949219
5   0246sfps  (651) 484-2525
6     krd537  (832) 296-9644
7     blr478  (817) 498-8502
8     pdm395  (814) 769-6298
9   0366swiz  (806) 358-3596
10       rt6  (512) 292-3067


In [34]:
for i in range(len(df_prfrPh1)): 
    #df_ph_csv.iloc[i,2]=re.findall(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]',df_ph_csv.iloc[i,1])
    df_prfrPh1.loc[i,'telephone']=df_prfrPh1.iloc[i,1].replace("(","").replace(")","").replace("-","").replace("/","").replace(" ","")
df_prfrPh1.to_csv(r'C:\Users\le5752\Documents\DB_Python\ph3.csv', index = False)
print(df_prfrPh1.head(11))

         EID           Phone   telephone
0     to3269  (281) 772-0155  2817720155
1   0563numt      5124967336  5124967336
2      ksn99  (361) 865-2686  3618652686
3    jme3234      9152745699  9152745699
4   0343rtia      4258949219  4258949219
5   0246sfps  (651) 484-2525  6514842525
6     krd537  (832) 296-9644  8322969644
7     blr478  (817) 498-8502  8174988502
8     pdm395  (814) 769-6298  8147696298
9   0366swiz  (806) 358-3596  8063583596
10       rt6  (512) 292-3067  5122923067


In [35]:
df_prfrPh1['EID'] = df_prfrPh1['EID'].str.upper()
df_prfrPh1.to_csv(r'C:\Users\le5752\Documents\DB_Python\ph3.csv', index = False)
print(df_prfrPh1.head(11))

         EID           Phone   telephone
0     TO3269  (281) 772-0155  2817720155
1   0563NUMT      5124967336  5124967336
2      KSN99  (361) 865-2686  3618652686
3    JME3234      9152745699  9152745699
4   0343RTIA      4258949219  4258949219
5   0246SFPS  (651) 484-2525  6514842525
6     KRD537  (832) 296-9644  8322969644
7     BLR478  (817) 498-8502  8174988502
8     PDM395  (814) 769-6298  8147696298
9   0366SWIZ  (806) 358-3596  8063583596
10       RT6  (512) 292-3067  5122923067


In [36]:
raw_eid = df_prfrPh1['EID'].dropna()
eid = raw_eid.tolist()
#print(len(eid))
# How many elements each list should have 
n = 1000
pid = {}
for i in range((len(eid) + n - 1) // n ):
    #globals()["eid" + str(i)] = eid[i * n:(i + 1) * n]
    pid[i] = eid[i * n:(i + 1) * n]

In [82]:
import cx_Oracle
connection = cx_Oracle.connect("xxxx/Zyyy@ea.y.z.com:1525/z.x.y.com")
cursor = connection.cursor()
df_ph_db = pd.DataFrame(columns = ['EID','Area','phone'])
for i in range(2):
    cursor.execute("select UT_EID, vi_phone_area_code, vi_phone_nbr from vi_vips_main m, vi_vips_phone_info p where m.ALUMNI_ID = p.ALUMNI_ID AND m.ut_eid IN %s" % str(tuple(pid[i])).replace(',)',')'))
    results = cursor.fetchall()
    df1 = pd.DataFrame(results, columns = ['EID','Area','phone'])
    df_ph_db = df_ph_db.append(df1,ignore_index=True)
cursor.close()
connection.close()

In [83]:
df_ph_db["telephone"] = df_ph_db["Area"].astype(str) + df_ph_db["phone"].astype(str)

In [84]:
#df_ph_db["telephone"] = df_ph_db["Area"].astype(str) + df_ph_db["phone"].astype(str)
for i in range(len(df_ph_db)):
    nan_rows = df_ph_db[df_ph_db['Area'].isnull()]
#print(nan_rows)
nan_rows["telephone"] = nan_rows["phone"].astype(str)
#print(nan_rows)

u_EID = nan_rows['EID']
u_EID = u_EID.drop_duplicates()
uni_EID = u_EID.tolist()

df_ph_db = df_ph_db[~df_ph_db['EID'].isin(uni_EID)]
df_ph_db = df_ph_db.append(nan_rows,ignore_index=True)

df_ph_db.to_csv(r'C:\Users\le5752\Documents\DB_Python\ph_out_sep3.csv', index = False)


           EID  Area         phone         telephone
585      GB343  None      14792378      None14792378
588      GM244  None    8677130601    None8677130601
589      GM244  None    8677134294    None8677134294
611      GM244  None       5281293       None5281293
815    KIKUCHI  None       8874967       None8874967
850      LC599  None  525555135037  None525555135037
851      LC599  None  525523095834  None525523095834
852      LC599  None  525555135037  None525555135037
917    MASONEE  None          6108          None6108
1528  0976QTMU  None        281781        None281781
1745    DP8938  None     602743945     None602743945
1753   DL34587  None     226194368     None226194368
2012     LZC59  None        795575        None795575
2070     MOY68  None    8098001001    None8098001001
2084   MM89285  None    1305712016    None1305712016
2093     PANGJ  None  886222197491  None886222197491
2123   PRM2239  None     910593113     None910593113
2360     WEIY1  None  862157742934  None862157

C:\Users\le5752\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [85]:
df_phone = pd.DataFrame()
for i in range(len(df_prfrPh1)):
    id_csv = df_prfrPh1.iloc[i,0]
    #print(id_csv)
    ph_csv = df_prfrPh1.iloc[i,2]
    #print(ph_csv)
    ph_db = (df_ph_db.loc[df_ph_db['EID'] == id_csv]).iloc[:,3]
    #print(ph_db) 
    ph_db = {*ph_db}
    #print(ph_db, ph_csv)
    ph_csv = {ph_csv}
    diff = (ph_csv - ph_db)
    if len(diff) > 0:
        df_phone = df_phone.append({'EID':id_csv, 'telephone':df_prfrPh1.iloc[i,2]}, ignore_index=True)

In [86]:
df_phone = df_phone.drop_duplicates()

df_phone['DB_phone'] = ''
for i in range(len(df_phone)):
    id_csv = df_phone.iloc[i,0]
    ph_db = (df_ph_db.loc[df_ph_db['EID'] == id_csv]).iloc[:,3]
    df_phone.iloc[i,2] = ph_db.values.tolist()
    
df_phone['telephone'].dropna()
#print(df_phone['telephone'].dropna())
df_phone.to_csv(r'C:\Users\le5752\Documents\DB_Python\ph.csv', index = False)